# Phase 0: Protocol Validation for Brain Templates¶

This notebook contains results validating the AFID protocol on five openly available macaque templates (D99, INIA19, MNI, NMTv1.3, and Yerkes19). At this stage, the protocol and specific instructions for placing the templates is still in flux (thus Phase 0). This project was based on work accomplished at the BrainWeb sprint in April 2020.

At this point, we'd like to be more sensitive to discrepancies in rater placements so an outlier is defined as a value of > 1.5 mm (3 voxels) from the mean.

The first step is to initialize the variables, define useful functions, and load all the raw fcsv data into df_raters.


In [1]:
# initialize libraries
library(plyr)
library(digest)
library(reshape2)
library(ggplot2)
#library("plot3D")

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
# useful functions

# calculate the distance between two sets of coordinates
dist3D <- function(coord1, coord2) { # vector X,Y,Z
        xdist <- coord1[1] - coord2[1] # could also write as coord1$X, etc.
        ydist <- coord1[2] - coord2[2]
        zdist <- coord1[3] - coord2[3]
        return(as.numeric(sqrt(xdist^2+ydist^2+zdist^2)))
}

# calculate the pairwise distance between an array of 3D coordinates
pairwise_dist3D <- function(temp_coords) { # labeled X,Y,Z
        N <- length(temp_coords$X)
        dist_vec <- rep(0,N) # create vector
        sum_dist <- 0 # initialize to zero
        count <- 0
        for (i in 1:(N-1)) {
                for (j in (i+1):N) {
                        if (i != j) {
                                count <- count + 1
                                first_coord <- temp_coords[i,]
                                second_coord <- temp_coords[j,]
                                curr_dist <- dist3D(first_coord, second_coord)
                                sum_dist <- sum_dist + curr_dist
                                dist_vec[count] <- curr_dist
                        }
                }
        }
        return(c(as.numeric(mean(dist_vec)),as.numeric(sd(dist_vec))))
}

In [3]:
# initialize variables and load in raw fcsv data into df_raters
setwd('~/Documents/GitHub/afids-macaca/data/PHASE0_input_afid/')

df_afids <- read.table('~/Documents/GitHub/afids-macaca/etc/afids.csv', sep=",", header=TRUE)

df_raters <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),rater=factor(),
                        template=factor(),mri_type=factor(),session=integer(),
                        name=character(),description=character(),stringsAsFactors = FALSE)
csv_files <- list.files('.', "*.fcsv")

for (i in 1:length(csv_files)) {
    curr_split <- unlist(strsplit(csv_files[i],"_"))
    if (length(curr_split)>1) { # extract name and session data
        rater_template <- curr_split[2]
        rater_mri_type <- curr_split[3]
        rater_name <- curr_split[4]
        rater_session <- as.numeric(unlist(strsplit(curr_split[5],"[.]"))[1])
    }
    curr_rater <- read.table(csv_files[i], header=FALSE, sep=",")
    df_rater <- data.frame(fid = 1:length(curr_rater$V1))

    df_rater <- cbind(df_rater,X=curr_rater[2],Y=curr_rater[3],Z=curr_rater[4],rater=rater_name,
                    template=rater_template,mri_type=rater_mri_type,
                    session=rater_session,name=curr_rater[12],
                    description=curr_rater[13])
  
    df_rater <- rename(df_rater, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))
    df_raters <- rbind(df_raters,df_rater)
}

levels(df_raters$rater) <- 1:8

# Template Averages

For each template, we calculate the mean value for each afid point and store it in a separate .fcsv file so that it can be loaded back into 3D Slicer.

Deviation of the values by > 10 mm will be classified as an outlier.

In [4]:
# start by calculating mean coordinates
df_template_mean <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),
                        template=factor(), name=factor(),description=character(),stringsAsFactors = FALSE)
df_template_sd <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),
                        template=factor(), name=factor(),description=character(),stringsAsFactors = FALSE)

# iterate over each template and compute the mean and standard deviation
for (curr_template in levels(df_raters$template)) {
    for (i in 1:32) { # for each AFID32 point, calculate the mean
        df_subset <- subset(df_raters, fid == i & template == curr_template)
        curr_fid_name <- df_afids$name[i]
        curr_fid_desc <- df_afids$description[i]
        df_curr_fid <- data.frame(fid = i, X = mean(df_subset$X), Y = mean(df_subset$Y), Z = mean(df_subset$Z),
                        template=curr_template, name=curr_fid_name, description=curr_fid_desc)
        df_template_mean <- rbind(df_template_mean, df_curr_fid)
        df_curr_fid_sd <- data.frame(fid = i, X = sd(df_subset$X), Y = sd(df_subset$Y), Z = sd(df_subset$Z),
                        template=curr_template, name=curr_fid_name, description=curr_fid_desc)
        df_template_sd <- rbind(df_template_sd, df_curr_fid_sd)
    }
}

# TODO: in afids-analysis, we create separate .fcsv files for each template

In [5]:
# start by calculating mean coordinates
df_template_mean <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),
                        template=factor(), name=factor(),description=character(),stringsAsFactors = FALSE)
df_template_sd <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),
                        template=factor(), name=factor(),description=character(),stringsAsFactors = FALSE)

# iterate over each template and compute the mean and standard deviation
for (curr_template in levels(df_raters$template)) {
    for (i in 1:32) { # for each AFID32 point, calculate the mean
        df_subset <- subset(df_raters, fid == i & template == curr_template)
        curr_fid_name <- df_afids$name[i]
        curr_fid_desc <- df_afids$description[i]
        df_curr_fid <- data.frame(fid = i, X = mean(df_subset$X), Y = mean(df_subset$Y), Z = mean(df_subset$Z),
                        template=curr_template, name=curr_fid_name, description=curr_fid_desc)
        df_template_mean <- rbind(df_template_mean, df_curr_fid)
        df_curr_fid_sd <- data.frame(fid = i, X = sd(df_subset$X), Y = sd(df_subset$Y), Z = sd(df_subset$Z),
                        template=curr_template, name=curr_fid_name, description=curr_fid_desc)
        df_template_sd <- rbind(df_template_sd, df_curr_fid_sd)
    }
}

##########################################################################################
# d99
##########################################################################################
df_d99_mean <- subset(df_template_mean, template == 'd99')
df_d99_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_d99_mean$fid,sep="_"),x=df_d99_mean$X,y=df_d99_mean$Y,z=df_d99_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_d99_mean$fid,desc=df_d99_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE0_output_afid/d99_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_d99_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

##########################################################################################
# INIA19
##########################################################################################
df_inia19_mean <- subset(df_template_mean, template == 'inia19')
df_inia19_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_inia19_mean$fid,sep="_"),x=df_inia19_mean$X,y=df_inia19_mean$Y,z=df_inia19_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_inia19_mean$fid,desc=df_inia19_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE0_output_afid/inia19_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_inia19_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

##########################################################################################
# macaqueMNI
##########################################################################################
df_macaqueMNI_mean <- subset(df_template_mean, template == 'macaqueMNI')
df_macaqueMNI_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_macaqueMNI_mean$fid,sep="_"),x=df_macaqueMNI_mean$X,y=df_macaqueMNI_mean$Y,z=df_macaqueMNI_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_macaqueMNI_mean$fid,desc=df_macaqueMNI_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE0_output_afid/macaqueMNI_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_macaqueMNI_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

##########################################################################################
# nmtv1.3
##########################################################################################
df_nmtv1.3_mean <- subset(df_template_mean, template == 'nmtv1.3')
df_nmtv1.3_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_nmtv1.3_mean$fid,sep="_"),x=df_nmtv1.3_mean$X,y=df_nmtv1.3_mean$Y,z=df_nmtv1.3_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_nmtv1.3_mean$fid,desc=df_nmtv1.3_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE0_output_afid/nmtv1.3_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_nmtv1.3_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

##########################################################################################
# yerkes19
##########################################################################################
df_yerkes19_mean <- subset(df_template_mean, template == 'yerkes19')
df_yerkes19_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_yerkes19_mean$fid,sep="_"),x=df_yerkes19_mean$X,y=df_yerkes19_mean$Y,z=df_yerkes19_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_yerkes19_mean$fid,desc=df_yerkes19_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE0_output_afid/yerkes19_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_yerkes19_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

# Phase 0: Raw Data Analysis

Also classify extreme outliers, defined as >= 10 mm from the group mean

In [6]:
df_raters$mean_AFLE <- NA # mean AFID localization error
df_raters$outlier <- NA
df_raters$xdist <- NA
df_raters$ydist <- NA
df_raters$zdist <- NA

for (i in 1:dim(df_raters)[1]) {
    curr_rater <- df_raters[i,]
        
    mean_raters <- df_d99_mean[curr_rater$fid,] # just so it's set to something for now
    
    #determine current template in order to assign the appropriate mean rater
    if (curr_rater$template == 'd99') {
        mean_raters <- df_d99_mean[curr_rater$fid,]
    } else if (curr_rater$template == 'inia19') {
        mean_raters <- df_inia19_mean[curr_rater$fid,]
    } else if (curr_rater$template == 'macaqueMNI') {
        mean_raters <- df_macaqueMNI_mean[curr_rater$fid,]
    } else if (curr_rater$template == 'nmtv1.3') {
        mean_raters <- df_nmtv1.3_mean[curr_rater$fid,]
    } else if (curr_rater$template == 'yerkes19') {
        mean_raters <- df_yerkes19_mean[curr_rater$fid,]        
    } else {
        # unidentified template, ...
    }
    df_raters[i,]$xdist <- curr_rater$X - mean_raters$X
    df_raters[i,]$ydist <- curr_rater$Y - mean_raters$Y
    df_raters[i,]$zdist <- curr_rater$Z - mean_raters$Z
    curr_coords <- curr_rater[,2:4]
    mean_coords <- mean_raters[,2:4]
    df_raters[i,]$mean_AFLE <- dist3D(curr_coords, mean_coords)
    df_raters[i,]$outlier <- (df_raters[i,]$mean_AFLE > 1.5) # outliers > 1.5mm
}

In [7]:
# summary of findings

# Total
all_templates <- subset(df_raters, session > 0) # ignore session 0 which was from the group tutorial
num_outliers <- sum(subset(all_templates, outlier == TRUE)$outlier)
num_total <- length(all_templates$outlier)
sprintf( "Total: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
        mean(all_templates$mean_AFLE), sd(all_templates$mean_AFLE),
        num_outliers, num_total, (num_outliers/num_total)*100 )

# d99
curr_template <- subset(df_raters, session > 0 & template == 'd99')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "d99: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )

# inia19
curr_template <- subset(df_raters, session > 0 & template == 'inia19')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "inia19: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )

# macaqueMNI
curr_template <- subset(df_raters, session > 0 & template == 'macaqueMNI')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "macaqueMNI: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )

# nmtv1.3
curr_template <- subset(df_raters, session > 0 & template == 'nmtv1.3')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "nmtv1.3: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )

# yerkes19
curr_template <- subset(df_raters, session > 0 & template == 'yerkes19')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "yerkes19: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )


[1] "Total: 0.54 +/- 0.46 mm; Outliers: 35/800 (4.38%)"

[1] "d99: 0.59 +/- 0.47 mm; Outliers: 10/160 (6.25%)"

[1] "inia19: 0.52 +/- 0.50 mm; Outliers: 9/160 (5.62%)"

[1] "macaqueMNI: 0.48 +/- 0.36 mm; Outliers: 5/160 (3.12%)"

[1] "nmtv1.3: 0.53 +/- 0.51 mm; Outliers: 6/160 (3.75%)"

[1] "yerkes19: 0.57 +/- 0.46 mm; Outliers: 5/160 (3.12%)"

In [8]:
# summarize results for each fid and template

summary_all_df <- ddply(df_raters, .(fid), summarize, mean_total=mean(mean_AFLE), sd_total=sd(mean_AFLE), sum_outliers_total=sum(outlier))

summary_temp_df <- ddply(df_raters, .(fid,template), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), sum_outliers=sum(outlier))

# d99
summary_all_df$mean_d99 <- subset(summary_temp_df, template == "d99")$mean
summary_all_df$sd_d99 <- subset(summary_temp_df, template == "d99")$sd
summary_all_df$sum_outliers_d99 <- subset(summary_temp_df, template == "d99")$sum_outliers

# inia19
summary_all_df$mean_inia19 <- subset(summary_temp_df, template == "inia19")$mean
summary_all_df$sd_inia19 <- subset(summary_temp_df, template == "inia19")$sd
summary_all_df$sum_outliers_inia19 <- subset(summary_temp_df, template == "inia19")$sum_outliers

# macaqueMNI
summary_all_df$mean_macaqueMNI <- subset(summary_temp_df, template == "macaqueMNI")$mean
summary_all_df$sd_macaqueMNI <- subset(summary_temp_df, template == "macaqueMNI")$sd
summary_all_df$sum_outliers_macaqueMNI <- subset(summary_temp_df, template == "macaqueMNI")$sum_outliers

# nmtv1.3
summary_all_df$mean_nmtv1.3 <- subset(summary_temp_df, template == "nmtv1.3")$mean
summary_all_df$sd_nmtv1.3 <- subset(summary_temp_df, template == "nmtv1.3")$sd
summary_all_df$sum_outliers_nmtv1.3 <- subset(summary_temp_df, template == "nmtv1.3")$sum_outliers

# yerkes19
summary_all_df$mean_yerkes19 <- subset(summary_temp_df, template == "yerkes19")$mean
summary_all_df$sd_yerkes19 <- subset(summary_temp_df, template == "yerkes19")$sd
summary_all_df$sum_outliers_yerkes19 <- subset(summary_temp_df, template == "yerkes19")$sum_outliers

summary_all_df[,c(-1,-4,-7,-10,-13)] <- round(summary_all_df[,c(-1,-4,-7,-10,-13)],2)
summary_all_df

write.table(summary_all_df, file = "~/Documents/GitHub/afids-macaca/data/PHASE0_output_afid/PHASE0_template_validation_afid_AFLE.csv", row.names = FALSE, quote = FALSE, sep = ",")

fid,mean_total,sd_total,sum_outliers_total,mean_d99,sd_d99,sum_outliers_d99,mean_inia19,sd_inia19,sum_outliers_inia19,mean_macaqueMNI,sd_macaqueMNI,sum_outliers_macaqueMNI,mean_nmtv1.3,sd_nmtv1.3,sum_outliers_nmtv1.3,mean_yerkes19,sd_yerkes19,sum_outliers_yerkes19
1,0.13,0.06,0,0.12,0.03,0,0.14,0.10,0,0.14,0.04,0,0.12,0.08,0,0.14,0.05,0
2,0.16,0.08,0,0.24,0.10,0,0.10,0.03,0,0.15,0.07,0,0.17,0.10,0,0.15,0.04,0
3,0.31,0.17,0,0.40,0.27,0,0.31,0.09,0,0.26,0.07,0,0.21,0.13,0,0.37,0.19,0
4,0.27,0.12,0,0.25,0.14,0,0.32,0.12,0,0.22,0.11,0,0.27,0.15,0,0.31,0.07,0
5,0.22,0.11,0,0.18,0.11,0,0.23,0.11,0,0.19,0.09,0,0.17,0.04,0,0.32,0.13,0
6,0.70,0.51,2,0.85,0.43,1,0.50,0.36,0,0.52,0.33,0,0.62,0.46,0,1.00,0.83,1
7,0.74,0.55,2,0.95,0.56,1,0.55,0.32,0,0.58,0.30,0,0.53,0.36,0,1.09,0.91,1
8,0.60,0.29,0,0.62,0.48,0,0.70,0.22,0,0.49,0.20,0,0.71,0.21,0,0.49,0.25,0
9,0.49,0.25,0,0.41,0.27,0,0.38,0.24,0,0.46,0.27,0,0.71,0.16,0,0.50,0.23,0
10,0.48,0.43,2,0.76,0.59,1,0.81,0.51,1,0.34,0.20,0,0.13,0.05,0,0.34,0.16,0


In [9]:
ddply(df_raters, .(rater), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), sum_outliers=sum(outlier))

rater,mean,sd,sum_outliers
1,0.5209650,0.3969230,5
2,0.4306731,0.3359881,3
3,0.4088475,0.3965922,6
4,0.7107250,0.5464480,15
5,0.6234402,0.5375905,6


In [10]:
ddply(df_raters, .(template,rater), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), sum_outliers=sum(outlier))

template,rater,mean,sd,sum_outliers
d99,1,0.5988578,0.3897789,1
d99,2,0.4910849,0.4144809,1
d99,3,0.4997831,0.4729919,2
d99,4,0.7204024,0.5840240,5
d99,5,0.6603010,0.4764970,1
inia19,1,0.5548554,0.5467129,2
inia19,2,0.3754984,0.3292815,1
inia19,3,0.4410776,0.4986659,2
inia19,4,0.6453989,0.4871586,2
inia19,5,0.5994473,0.5923666,2


In [11]:
# sample of individual rater data > 1.5 mm from the mean
ddply(subset(df_raters, mean_AFLE > 1.5), .(fid,description), summarize, mean_AFLE = mean(mean_AFLE))

fid,description,mean_AFLE
6,R superior LMS,1.991895
7,L superior LMS,2.272010
10,culmen,1.720365
14,pineal gland,2.391597
27,R indusium griseum origin,2.142403
28,L indusium griseum origin,2.049865
29,R ventral occipital horn,2.046500
30,L ventral occipital horn,2.305469
32,L olfactory sulcal fundus,1.568520


In [12]:
subset(df_raters, mean_AFLE > 1.5 & rater == 1)

,fid,X,Y,Z,rater,template,mri_type,session,name,description,mean_AFLE,outlier,xdist,ydist,zdist
27,27,5.750,-20.789,6.664,1,d99,T1,1,27,R indusium griseum origin,1.687685,TRUE,-1.416918,-0.08944719,0.9124811
187,27,5.528,-21.388,4.056,1,inia19,T1,1,27,R indusium griseum origin,2.428750,TRUE,-1.889563,-0.77544248,1.3141789
188,28,-5.974,-21.324,3.828,1,inia19,T1,1,28,L indusium griseum origin,2.234033,TRUE,1.876163,-0.65132606,1.0230791
347,27,4.500,-19.130,4.686,1,macaqueMNI,T1,1,27,R indusium griseum origin,2.060235,TRUE,-1.656435,-0.72105501,0.9903899
348,28,-5.249,-19.268,4.361,1,macaqueMNI,T1,1,28,L indusium griseum origin,1.708721,TRUE,1.214525,-0.87383875,0.8252665


In [13]:
subset(df_raters, mean_AFLE > 1.5 & rater == 2)

,fid,X,Y,Z,rater,template,mri_type,session,name,description,mean_AFLE,outlier,xdist,ydist,zdist
42,10,-0.2286675,-29.03934,7.314584,2,d99,T1,1,10,culmen,1.806126,TRUE,-0.05813403,-1.792871,-0.21053265
202,10,-0.1560955,-28.26979,3.865907,2,inia19,T1,1,10,culmen,1.634603,TRUE,-0.05027642,1.576971,-0.42727434
526,14,-0.2671972,-13.74560,3.764306,2,nmtv1.3,T1,1,14,pineal gland,1.924758,TRUE,-0.02075773,1.922922,0.08144507


In [14]:
subset(df_raters, mean_AFLE > 1.5 & rater == 3)

,fid,X,Y,Z,rater,template,mri_type,session,name,description,mean_AFLE,outlier,xdist,ydist,zdist
91,27,8.741,-20.259,4.580,3,d99,T1,1,27,R indusium griseum origin,2.011039,TRUE,1.574082,0.4405528,-1.1715189
92,28,-8.469,-20.337,4.826,3,d99,T1,1,28,L indusium griseum origin,1.707772,TRUE,-1.285940,0.4237015,-1.0408274
251,27,8.938,-19.822,1.616,3,inia19,T1,1,27,R indusium griseum origin,2.050410,TRUE,1.520437,0.7905575,-1.1258211
252,28,-9.636,-19.846,1.603,3,inia19,T1,1,28,L indusium griseum origin,2.305910,TRUE,-1.785837,0.8266739,-1.2019209
411,27,7.550,-18.141,2.956,3,macaqueMNI,T1,1,27,R indusium griseum origin,1.600263,TRUE,1.393565,0.2679450,-0.7396101
571,27,9.318,-19.346,4.839,3,nmtv1.3,T1,1,27,R indusium griseum origin,1.897667,TRUE,1.705533,0.5170334,-0.6518984


In [15]:
subset(df_raters, mean_AFLE > 1.5 & rater == 4)

,fid,X,Y,Z,rater,template,mri_type,session,name,description,mean_AFLE,outlier,xdist,ydist,zdist
102,6,7.051,-15.236,-1.839,4,d99,T1,1,6,R superior LMS,1.561850,TRUE,0.395336602,-0.1797033,1.5002641
103,7,-7.364,-15.300,-1.440,4,d99,T1,1,7,L superior LMS,1.899020,TRUE,-0.421939379,-0.1871972,1.8420641
123,27,8.938,-20.972,4.711,4,d99,T1,1,27,R indusium griseum origin,2.072110,TRUE,1.771081813,-0.2724472,-1.0405189
124,28,-8.843,-20.979,4.927,4,d99,T1,1,28,L indusium griseum origin,1.919982,TRUE,-1.659939650,-0.2182985,-0.9398274
128,32,-5.202,13.484,3.419,4,d99,T1,1,32,L olfactory sulcal fundus,1.568520,TRUE,0.013447347,1.2635298,0.9292826
283,27,9.337,-20.052,1.556,4,inia19,T1,1,27,R indusium griseum origin,2.324787,TRUE,1.919437045,0.5605575,-1.1858211
284,28,-9.202,-20.250,2.052,4,inia19,T1,1,28,L indusium griseum origin,1.604060,TRUE,-1.351837491,0.4226739,-0.7529209
444,28,-7.639,-17.616,2.615,4,macaqueMNI,T1,1,28,L indusium griseum origin,1.683754,TRUE,-1.175475464,0.7781613,-0.9207335
445,29,14.300,-22.781,1.325,4,macaqueMNI,T1,1,29,R ventral occipital horn,1.544868,TRUE,0.530608667,-1.1337491,0.9053647
605,29,15.323,-25.782,3.528,4,nmtv1.3,T1,1,29,R ventral occipital horn,2.548133,TRUE,-0.002768165,-2.2788753,1.1400439


In [16]:
subset(df_raters, mean_AFLE > 1.5 & rater == 5)

,fid,X,Y,Z,rater,template,mri_type,session,name,description,mean_AFLE,outlier,xdist,ydist,zdist
155,27,5.348,-20.655,6.862,5,d99,T1,1,27,R indusium griseum origin,2.131576,TRUE,-1.8189182,0.04455281,1.1104811
315,27,5.348,-21.404,4.101,5,inia19,T1,1,27,R indusium griseum origin,2.599392,TRUE,-2.0695630,-0.79144248,1.3591789
316,28,-5.935,-21.370,4.044,5,inia19,T1,1,28,L indusium griseum origin,2.385252,TRUE,1.9151625,-0.69732606,1.2390791
635,27,4.961,-20.165,6.487,5,nmtv1.3,T1,1,27,R indusium griseum origin,2.848452,TRUE,-2.6514668,-0.30196660,0.9961016
636,28,-4.836,-20.175,6.425,5,nmtv1.3,T1,1,28,L indusium griseum origin,3.149624,TRUE,2.9730904,-0.25767185,1.0072083
782,14,-0.230,-15.002,2.558,5,yerkes19,T1,1,14,pineal gland,2.858437,TRUE,0.1339099,2.75205918,0.7608563


# Post-QC analysis TODO